In [1]:
from langchain.pydantic_v1 import BaseModel, Field

class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

In [2]:
from langchain_core.tools import StructuredTool

# Define the functions for each operation
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b

def divide(a: int, b: int) -> float:
    """Divide two numbers."""
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

# Define the asynchronous versions of the functions
async def aadd(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

async def asubtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b

async def adivide(a: int, b: int) -> float:
    """Divide two numbers."""
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

async def amultiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


In [3]:
add_tool = StructuredTool.from_function(
    func=add,
    name="add",
    description="Add 2 numbers",
    args_schema=CalculatorInput,
    return_direct=False,
    coroutine=aadd
)

subtract_tool = StructuredTool.from_function(
    func=subtract,
    name="subtract",
    description="Subtract 2 numbers",
    args_schema=CalculatorInput,
    return_direct=False,
    coroutine=asubtract
)


multiply_tool = StructuredTool.from_function(
    func=multiply,
    name="multiply",
    description="Multiply 2 numbers",
    args_schema=CalculatorInput,
    return_direct=False,
    coroutine=amultiply
)


divide_tool = StructuredTool.from_function(
    func=divide,
    name="divide",
    description="Divide 2 numbers",
    args_schema=CalculatorInput,
    return_direct=False,
    coroutine=adivide
)




In [4]:

# Example usage
print(add_tool.invoke({"a": 2, "b": 3}))  # Output: 5
print(await subtract_tool.ainvoke({"a": 10, "b": 4}))  # Output: 6
print(divide_tool.invoke({"a": 8, "b": 2}))  # Output: 4.0
print(await multiply_tool.ainvoke({"a": 2, "b": 5}))  # Output: 10


5
6
4.0
10


In [5]:
tools = [add_tool, subtract_tool, multiply_tool, divide_tool]

In [6]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Code\Github\LangChain-Basics\venv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

# Calling the tool

Now, let's get the model to call a tool. We'll add it to a list of messages that we'll treat as conversation history:



In [8]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_keC1LQm8tm5cXgRgBiTquHjM'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_CnmddQPcd5z4e6TxbOQcqELX'}]


In [9]:
ai_msg.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_keC1LQm8tm5cXgRgBiTquHjM'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_CnmddQPcd5z4e6TxbOQcqELX'}]

In [12]:
from langchain_core.messages import ToolMessage

for tool_call in ai_msg.tool_calls:
    tool_name = tool_call['name']
    tool_args = tool_call['args']
    tool_call_id = tool_call['id']
    
    # Select the appropriate tool
    selected_tool = {"add": add_tool, "multiply": multiply_tool}.get(tool_name.lower())
    
    if selected_tool:
        # Invoke the tool and get the result
        result = selected_tool.invoke(tool_args)
        
        # Create a ToolMessage instance
        tool_msg = ToolMessage(content=result, name=tool_name, tool_call_id=tool_call_id)
        
        # Append the ToolMessage to the messages list
        messages.append(tool_msg)
    else:
        print(f"Tool '{tool_name}' not found.")

In [13]:
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_keC1LQm8tm5cXgRgBiTquHjM', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_CnmddQPcd5z4e6TxbOQcqELX', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 206, 'total_tokens': 256}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_507c9469a1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5c6cb102-721c-4785-8482-7e952b660b97-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_keC1LQm8tm5cXgRgBiTquHjM'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_CnmddQPcd5z4e6TxbOQcqELX'}]),
 ToolMessage(content='36', name='multiply', tool_call_id='call_keC1LQm8tm5cXgRgBiTquHjM'),
 ToolMessage(content='60', name='add', tool_call_id='call_CnmddQPcd5z4e6Tx

In [14]:
llm_with_tools.invoke(messages)

AIMessage(content='The result of \\(3 \\times 12\\) is 36, and \\(11 + 49\\) equals 60.', response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 272, 'total_tokens': 300}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-92c36135-c0bb-4663-9650-2a9f4675e785-0')